In [2]:
import agentpy as ap
from owlready2 import *
import random

In [6]:
#onto = get_ontology("file:///Multi Agentes/driving_onto.rdf").load()
onto = get_ontology("file:///AgentPy/Activity-4-Two-way-communication/driving_onto.rdf").load()

with onto:
    class Entity(Thing):
        pass

    class Agent(Entity):
        pass

    class Driver(Agent):
        pass

    class Vehicle(Agent):
        pass

    class Location(Entity):
        pass

    class PointA(Location):
        pass

    class PointB(Location):
        pass

    class is_at(ObjectProperty):
        domain = [Agent]
        range = [Location]

    class has_destination(ObjectProperty):
        domain = [Driver]
        range = [Location]

    print(onto.Entity)
    print(onto.classes())
    print(list(onto.classes()))
# Save the ontology if needed
# onto.save(file="path_to_save.owl", format="rdfxml")


driving_onto.Entity
<generator object _GraphManager.classes at 0x000001D6161C7300>
[driving_onto.Driver, driving_onto.Location, driving_onto.Agent, driving_onto.Entity, driving_onto.Vehicle, driving_onto.PointA, driving_onto.PointB]


In [3]:
class DriverAgent(ap.Agent):
    def setup(self):
        self.beliefs = {'current_location': (0, 0), 'path_index': 0}
        self.path = [(1, 0), (1, 0), (0, 1), (0, 1), (-1, 0)]  # Sample path
        self.destination = 'B'

    def see(self, model):
        # Pass the model instead of the grid
        self.beliefs['current_location'] = model.get_agent_location(self)

    def decide(self):
        if self.destination == 'B':
            if self.beliefs['path_index'] < len(self.path):
                self.intentions = self.path[self.beliefs['path_index']]
                self.beliefs['path_index'] += 1
            else:
                self.destination = 'A'
        elif self.destination == 'A':
            # Logic to return to Point A (reverse path)
            pass

    def send_message(self, vehicle):
        if self.intentions:
            vehicle.receive_message(self.intentions)



In [4]:
class VehicleAgent(ap.Agent):
    def setup(self):
        self.position = (0, 0)  # Starting at Point A

    def receive_message(self, message):
        self.move(message)

    def move(self, direction):
        dx, dy = direction
        new_x = self.position[0] + dx
        new_y = self.position[1] + dy
        self.position = (new_x, new_y)

    def inform(self, driver):
        # For now, let's just print the information
        print(f'Moved to {self.position}')


In [5]:
class TransportModel(ap.Model):
    def setup(self):
        self.grid = ap.Grid(self, self.p['worldSize'], track_empty=True)
        self.driver = DriverAgent(self)
        self.vehicle = VehicleAgent(self)

        # Provide a list of positions for each agent
        self.grid.add_agents([self.driver], [(0, 0)])  # Wrap the position in a list
        self.grid.add_agents([self.vehicle], [(0, 0)])  # Wrap the position in a list

    def step(self):
        self.driver.see(self)  # Pass the model itself
        self.driver.decide()
        if self.driver.intentions:
            self.driver.send_message(self.vehicle)
            self.vehicle.inform(self.driver)

    def get_agent_location(self, agent):
        return self.grid.positions[agent]


In [6]:
r = random.random()

parameters = {
    "driverAgent": 1,        # Number of Driver Agents
    "vehicleAgent": 1,       # Number of Vehicle Agents
    "worldSize": (10, 10),   # Size of the grid world
    "steps": 50,             # Maximum number of steps to simulate
    "seed": 13 * r           # Seed for random variables
}
model = TransportModel(parameters)
results = model.run()


Moved to (1, 0)
Completed: 1 stepsMoved to (2, 0)
Completed: 2 stepsMoved to (2, 1)
Completed: 3 stepsMoved to (2, 2)
Completed: 4 stepsMoved to (1, 2)
Completed: 5 stepsMoved to (0, 2)
Completed: 6 stepsMoved to (-1, 2)
Completed: 7 stepsMoved to (-2, 2)
Completed: 8 stepsMoved to (-3, 2)
Completed: 9 stepsMoved to (-4, 2)
Completed: 10 stepsMoved to (-5, 2)
Completed: 11 stepsMoved to (-6, 2)
Completed: 12 stepsMoved to (-7, 2)
Completed: 13 stepsMoved to (-8, 2)
Completed: 14 stepsMoved to (-9, 2)
Completed: 15 stepsMoved to (-10, 2)
Completed: 16 stepsMoved to (-11, 2)
Completed: 17 stepsMoved to (-12, 2)
Completed: 18 stepsMoved to (-13, 2)
Completed: 19 stepsMoved to (-14, 2)
Completed: 20 stepsMoved to (-15, 2)
Completed: 21 stepsMoved to (-16, 2)
Completed: 22 stepsMoved to (-17, 2)
Completed: 23 stepsMoved to (-18, 2)
Completed: 24 stepsMoved to (-19, 2)
Completed: 25 stepsMoved to (-20, 2)
Completed: 26 stepsMoved to (-21, 2)
Completed: 27 stepsMoved to (-22, 2)
Completed: 28